In [29]:
#Example 
n = 2
p = 2
X = np.random.normal(0, 1, (n, p))
y = np.random.normal(0, 1, n)
X_transpose = X.T
beta= [1,2]
beta_init= [1,2]
eta_j=1
j=1
eta = [1,2]
sigma_sq=1
num_iterations=1000

In [14]:
import numpy as np

In [30]:
# Fonction pour calculer mu_j
def compute_mu_j(beta, X, y, eta_j, sigma_sq, j):
    n = X.shape[0]
    mu_j = np.sum(X[:, j] * (y - np.dot(X, beta) + X[:, j] * beta[j])) / (X[:, j].dot(X[:, j]) / eta_j + 1 / sigma_sq)
    return mu_j

print(compute_mu_j(beta, X, y, eta_j, sigma_sq, j))

-0.4341533162039672


In [31]:
# Fonction pour calculer sigma_j^2
def compute_sigma_j_sq(X, eta_j, sigma_sq, j):
    sigma_j_sq = 1 / (X[:, j].dot(X[:, j]) / eta_j + 1 / sigma_sq)
    return sigma_j_sq

print(compute_sigma_j_sq(X, eta_j, sigma_sq, j))

0.23757591630456587


In [34]:
# Fonction pour l'échantillonnage de Gibbs
def gibbs_sampling(X, y, beta_init, eta, sigma_sq, num_iterations):
    num_features = X.shape[1]
    
    # Boucle sur le nombre d'itérations
    for t in range(num_iterations):
        beta_new = beta_init
        
        # Boucle sur chaque coordonnée beta_j
        for j in range(num_features):
            mu_j = compute_mu_j(beta_new, X, y, eta[j], sigma_sq, j)
            sigma_j_sq = compute_sigma_j_sq(X, eta[j], sigma_sq, j)
            beta_new[j] = np.random.normal(mu_j, np.sqrt(sigma_j_sq))
        
        # Mettre à jour les coordonnées beta pour l'itération suivante
        beta_current = beta_new
    
    return beta_current

print(gibbs_sampling(X, y, beta_init, eta, sigma_sq, num_iterations))

TypeError: 'int' object is not subscriptable